In [1]:
import pandas as pd
from pybedtools import BedTool
from pybedtools.featurefuncs import TSS
import os

In [1]:
import sys
args = sys.argv  # a list of the arguments provided (str)
print("processing eqtl", args)
rawBed= args[1]
file = args[2]
fileHelp = args[3]

processing Chiapet ['/storage/bin/anaconda3/envs/yudeep/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/yudeep_rajbhandari1@baylor.edu/.local/share/jupyter/runtime/kernel-b706c871-6c2b-4f98-895b-1762e49f5631.json']


In [2]:
df1 = pd.read_csv('GTEx_Analysis_v8_eQTL/'+file, sep='\t')

In [3]:
df1.head()

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta
0,chr1_64764_C_T_b38,ENSG00000227232.5,35211,47,48,0.062016,0.000003,0.600208,0.126761,0.000299,0.000002,0.001462
1,chr1_665098_G_A_b38,ENSG00000227232.5,635545,87,91,0.117571,0.000002,0.468369,0.096095,0.000299,0.000002,0.001462
2,chr1_666028_G_A_b38,ENSG00000227232.5,636475,76,80,0.103359,0.000011,0.450967,0.101154,0.000299,0.000002,0.001462
3,chr1_64931_G_A_b38,ENSG00000268903.1,-70964,48,53,0.068475,0.000065,0.461347,0.113963,0.000259,0.000007,0.005910
4,chr1_108826_G_C_b38,ENSG00000268903.1,-27069,29,29,0.037468,0.000007,0.737364,0.161353,0.000259,0.000007,0.005910


In [4]:
def startPosition(df):
    return df.variant_id.split('_')[1]

In [5]:
def endPosition(df):
    return int(df.start)+1

In [6]:
def chrom(df):
    return df.variant_id.split('_')[0]

In [7]:
def geneIDNorm(df):
    return df.gene_id.split('.')[0]

In [8]:
df1['start'] = df1.apply(startPosition,axis=1)
df1['end'] = df1.apply(endPosition,axis = 1)
df1['chrom'] = df1.apply(chrom,axis=1)

In [9]:
df2 = df1[['chrom','start','end','gene_id','pval_nominal']]

In [10]:
df2.head()

,chrom,start,end,gene_id,pval_nominal
0,chr1,14677,14678,ENSG00000228463.9,1.226320e-06
1,chr1,591154,591155,ENSG00000225630.1,1.439870e-05
2,chr1,1449049,1449050,ENSG00000225630.1,5.029090e-07
3,chr1,1624723,1624724,ENSG00000225630.1,2.697290e-05
4,chr1,915107,915108,ENSG00000229376.3,3.795720e-06


In [11]:
df2 = df2.sort_values(by=['chrom','end'],ascending=True)

In [12]:
new_header = df2.iloc[0] 
df2 = df2[1:] 
df2.columns = new_header 


In [13]:
df2.head(10)

,chr1,14677,14678,ENSG00000228463.9,1.22632e-06
5,chr1,267754,267755,ENSG00000228327.3,4.397950e-05
1,chr1,591154,591155,ENSG00000225630.1,1.439870e-05
9,chr1,727034,727035,ENSG00000228794.8,3.880630e-07
10,chr1,727242,727243,ENSG00000228794.8,1.347760e-06
11,chr1,727477,727478,ENSG00000228794.8,2.645260e-05
12,chr1,758351,758352,ENSG00000228794.8,1.119820e-06
13,chr1,758443,758444,ENSG00000228794.8,7.161080e-07
82,chr1,769577,769578,ENSG00000225880.5,1.860920e-07
83,chr1,769579,769580,ENSG00000225880.5,1.860920e-07
84,chr1,775571,775572,ENSG00000225880.5,1.777780e-08


In [14]:
df2.to_csv('temp1.bed', sep="\t",index=False)

In [15]:
df3 = pd.read_csv('temp1.bed',sep = '\t')

In [16]:
df3.head()

,chr1,14677,14678,ENSG00000228463.9,1.22632e-06
0,chr1,267754,267755,ENSG00000228327.3,4.397950e-05
1,chr1,591154,591155,ENSG00000225630.1,1.439870e-05
2,chr1,727034,727035,ENSG00000228794.8,3.880630e-07
3,chr1,727242,727243,ENSG00000228794.8,1.347760e-06
4,chr1,727477,727478,ENSG00000228794.8,2.645260e-05


In [17]:
def getClosestgeneWithinDistance(enhancer,genes,distance,num):
    snp = BedTool(enhancer)
    gene = BedTool(genes)
    gene.sort()  
    nearby = snp.closest(gene, d=True,k=num, output='tempEQTL.bed')
    dfSNP = pd.read_csv(enhancer, sep='\t', header=None)
    dfGene = pd.read_csv(genes, sep='\t', header=None)
    df1 = pd.read_csv('tempEQTL.bed', sep='\t', header=None)
    df1 = df1[(df1[len(df1.columns)-1].between(-distance, distance))]
    
    df1.drop(columns=df1.columns[5:len(dfSNP.columns)], 
        axis=1, 
        inplace=True)
    print(df1.head())
    df1.drop(columns=df1.columns[9:len(dfGene.columns)+4], 
        axis=1, 
        inplace=True)
    os.remove('tempEQTL.bed')
    return df1

In [18]:
allBed = getClosestgeneWithinDistance('temp1.bed',rawBed,1000000,1)

     0       1       2                  3             4     5       6   \
0  chr1   14677   14678  ENSG00000228463.9  1.226320e-06  chr1  104896   
1  chr1  267754  267755  ENSG00000228327.3  4.397950e-05  chr1  267925   
2  chr1  591154  591155  ENSG00000225630.1  1.439870e-05  chr1  586036   
3  chr1  727034  727035  ENSG00000228794.8  3.880630e-07  chr1  727121   
4  chr1  727242  727243  ENSG00000228794.8  1.347760e-06  chr1  727121   

       7             8   9  10      11      12           13            14  \
0  105048  EH38E2776520   0  .  104896  105048  140,140,140  Unclassified   
1  268171  EH38E2776528   0  .  267925  268171  140,140,140  Unclassified   
2  586264  EH38E2776532   0  .  586036  586264  140,140,140  Unclassified   
3  727350  EH38E2776536   0  .  727121  727350  140,140,140  Unclassified   
4  727350  EH38E2776536   0  .  727121  727350  140,140,140  Unclassified   

                                    15     16  
0  Missing-data/Partial-classification  9021

In [19]:
allBed.head()

,0,1,2,3,4,5,6,7,8,15,16
0,chr1,14677,14678,ENSG00000228463.9,1.226320e-06,chr1,104896,105048,EH38E2776520,Missing-data/Partial-classification,90219
1,chr1,267754,267755,ENSG00000228327.3,4.397950e-05,chr1,267925,268171,EH38E2776528,Missing-data/Partial-classification,171
2,chr1,591154,591155,ENSG00000225630.1,1.439870e-05,chr1,586036,586264,EH38E2776532,Missing-data/Partial-classification,4891
3,chr1,727034,727035,ENSG00000228794.8,3.880630e-07,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,87
4,chr1,727242,727243,ENSG00000228794.8,1.347760e-06,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,0


In [20]:
len(allBed)

629688

In [21]:
allBed = allBed[allBed[16] <= 100 ]

In [22]:
allBed.head()

,0,1,2,3,4,5,6,7,8,15,16
3,chr1,727034,727035,ENSG00000228794.8,3.880630e-07,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,87
4,chr1,727242,727243,ENSG00000228794.8,1.347760e-06,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,0
13,chr1,778534,778535,ENSG00000225880.5,4.881370e-11,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,36
14,chr1,778639,778640,ENSG00000225880.5,2.996770e-07,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,0
15,chr1,778705,778706,ENSG00000225880.5,2.653800e-08,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,0


In [23]:
len(allBed)

129732

In [24]:
allBed = allBed.sort_values(by=[15,0],ascending=True)

In [25]:
allBed.head()

,0,1,2,3,4,5,6,7,8,15,16
3,chr1,727034,727035,ENSG00000228794.8,3.880630e-07,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,87
4,chr1,727242,727243,ENSG00000228794.8,1.347760e-06,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,0
13,chr1,778534,778535,ENSG00000225880.5,4.881370e-11,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,36
14,chr1,778639,778640,ENSG00000225880.5,2.996770e-07,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,0
15,chr1,778705,778706,ENSG00000225880.5,2.653800e-08,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,0


In [26]:
df4 = pd.read_csv('GTEx_Analysis_v8_eQTL/'+fileHelp,sep = '\t')

In [27]:
df4[(df4['gene_chr'] =='chr1') & (df4['gene_start'] ==147632194) & (df4['gene_end']==147632224)]

,gene_id,gene_name,gene_chr,gene_start,gene_end,strand,num_var,beta_shape1,beta_shape2,true_df,...,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold,log2_aFC,log2_aFC_lower,log2_aFC_upper


In [28]:
df4.head()

,gene_id,gene_name,gene_chr,gene_start,gene_end,strand,num_var,beta_shape1,beta_shape2,true_df,...,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold,log2_aFC,log2_aFC_lower,log2_aFC_upper
0,ENSG00000227232.5,WASH7P,chr1,14410,29553,-,1364,1.02638,311.223,155.695,...,0.000432,0.707137,0.196914,0.190051,0.186735,0.186765,0.000100,0.821879,0.459859,1.050541
1,ENSG00000268903.1,RP11-34P13.15,chr1,135141,135895,-,1863,1.05785,360.120,152.693,...,0.000151,0.643308,0.165928,0.090091,0.088980,0.116833,0.000098,1.346428,0.824443,1.796007
2,ENSG00000269981.1,RP11-34P13.16,chr1,137682,137965,-,1868,1.02634,333.712,151.167,...,0.000098,0.700484,0.175460,0.064194,0.065167,0.094553,0.000093,1.412633,0.832862,1.883766
3,ENSG00000241860.6,RP11-34P13.13,chr1,141474,173862,-,2066,1.04067,380.928,152.873,...,0.000159,0.890830,0.230550,0.097590,0.102113,0.128432,0.000087,1.528061,0.884833,1.969651
4,ENSG00000279457.4,RP11-34P13.18,chr1,185217,195411,-,2234,1.03882,382.742,150.872,...,0.000291,-0.696659,0.188241,0.186858,0.189284,0.188422,0.000086,-1.140384,-1.944668,-0.712864


In [29]:
finaldf = allBed.merge(df4,left_on=3,right_on='gene_id',how='left')

In [30]:
finaldf.head()

,0,1,2,3,4,5,6,7,8,15,...,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold,log2_aFC,log2_aFC_lower,log2_aFC_upper
0,chr1,727034,727035,ENSG00000228794.8,3.880630e-07,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,...,2.606100e-08,0.464257,0.079467,0.0002,6.057930e-05,2.355480e-04,0.000045,0.506442,0.306029,0.655156
1,chr1,727242,727243,ENSG00000228794.8,1.347760e-06,chr1,727121,727350,EH38E2776536,Missing-data/Partial-classification,...,2.606100e-08,0.464257,0.079467,0.0002,6.057930e-05,2.355480e-04,0.000045,0.506442,0.306029,0.655156
2,chr1,778534,778535,ENSG00000225880.5,4.881370e-11,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,...,2.105180e-11,0.860711,0.119816,0.0001,9.620600e-08,5.978450e-07,0.000046,1.071197,0.810703,1.253179
3,chr1,778639,778640,ENSG00000225880.5,2.996770e-07,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,...,2.105180e-11,0.860711,0.119816,0.0001,9.620600e-08,5.978450e-07,0.000046,1.071197,0.810703,1.253179
4,chr1,778705,778706,ENSG00000225880.5,2.653800e-08,chr1,778570,778919,EH38E2776539,Missing-data/Partial-classification,...,2.105180e-11,0.860711,0.119816,0.0001,9.620600e-08,5.978450e-07,0.000046,1.071197,0.810703,1.253179


In [31]:
finaldf = finaldf[[4,5,6,7,8,'gene_chr','gene_start','gene_end','gene_id',15]]

In [32]:
finaldf.head(10)

,4,5,6,7,8,gene_chr,gene_start,gene_end,gene_id,15
0,3.880630e-07,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,Missing-data/Partial-classification
1,1.347760e-06,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,Missing-data/Partial-classification
2,4.881370e-11,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
3,2.996770e-07,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
4,2.653800e-08,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
5,3.198490e-08,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
6,2.628110e-07,chr1,779026,779180,EH38E2776540,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
7,3.391980e-10,chr1,779222,779432,EH38E2776541,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
8,9.923830e-09,chr1,779222,779432,EH38E2776541,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification
9,4.881370e-11,chr1,779735,780028,EH38E2776542,chr1,826206,827522,ENSG00000225880.5,Missing-data/Partial-classification


In [33]:
finaldf[15] = finaldf[4]

In [34]:
finaldf.head()

,4,5,6,7,8,gene_chr,gene_start,gene_end,gene_id,15
0,3.880630e-07,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,3.880630e-07
1,1.347760e-06,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,1.347760e-06
2,4.881370e-11,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,4.881370e-11
3,2.996770e-07,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,2.996770e-07
4,2.653800e-08,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,2.653800e-08


In [35]:
finaldf = finaldf.rename(columns={5: "enahancer_chr", 6: "enhancer_start",7:"enhancer_end",8:"enhancer_id",15:"p-val"})

In [36]:
finaldf.drop(columns=finaldf.columns[0], 
        axis=1, 
        inplace=True)

In [37]:
finaldf.head()

,enahancer_chr,enhancer_start,enhancer_end,enhancer_id,gene_chr,gene_start,gene_end,gene_id,p-val
0,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,3.880630e-07
1,chr1,727121,727350,EH38E2776536,chr1,825138,859446,ENSG00000228794.8,1.347760e-06
2,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,4.881370e-11
3,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,2.996770e-07
4,chr1,778570,778919,EH38E2776539,chr1,826206,827522,ENSG00000225880.5,2.653800e-08


### Duplications issue
Here the duplication is because more than one eQtl exists in between enhancer start and end  pointing to the same gene in most occasions.

Solution: enahncerID and geneID should be unique.

In [38]:
finaldf['gene_id'] = finaldf.apply(geneIDNorm,axis=1)

In [39]:
finaldf = finaldf.drop_duplicates(subset=['enhancer_id', 'gene_id'], keep='last')

In [40]:
finaldf[finaldf['enhancer_id'].isin(['EH38E2836060','EH38E2836065','EH38E1382733','EH38E2836036','EH38E2836028','EH38E2835994'])]

,enahancer_chr,enhancer_start,enhancer_end,enhancer_id,gene_chr,gene_start,gene_end,gene_id,p-val
5495,chr1,147539962,147540170,EH38E2835994,chr1,147629652,147670496,ENSG00000162836,2.945460e-06
5497,chr1,147599385,147599535,EH38E2836028,chr1,147629652,147670496,ENSG00000162836,6.262620e-08
5498,chr1,147611213,147611484,EH38E2836036,chr1,147629652,147670496,ENSG00000162836,1.631070e-15
5501,chr1,147627788,147627955,EH38E1382733,chr1,147629652,147670496,ENSG00000162836,8.759060e-16
5503,chr1,147646291,147646558,EH38E2836060,chr1,147629652,147670496,ENSG00000162836,1.227200e-19
5504,chr1,147652317,147652651,EH38E2836065,chr1,147629652,147670496,ENSG00000162836,2.743550e-20


In [41]:
finaldf.tail()

,enahancer_chr,enhancer_start,enhancer_end,enhancer_id,gene_chr,gene_start,gene_end,gene_id,p-val
129727,chrX,155731820,155732165,EH38E2774677,chrX,154886349,154888061,ENSG00000277203,1.699620e-13
129728,chrX,155738988,155739150,EH38E3949987,chrX,154886349,154888061,ENSG00000277203,1.327740e-13
129729,chrX,155744429,155744642,EH38E3949988,chrX,154886349,154888061,ENSG00000277203,9.674760e-05
129730,chrX,155745804,155746105,EH38E2774681,chrX,154886349,154888061,ENSG00000277203,1.963390e-12
129731,chrX,155747551,155747849,EH38E3949990,chrX,154886349,154888061,ENSG00000277203,1.963390e-12


In [42]:
finaldf.to_csv('tempFinaleQTL.bed',sep='\t',header=None,index=False)